This exercise will require you to pull some data from https://data.nasdaq.com/ (formerly Quandl API).

As a first step, you will need to register a free account on the https://data.nasdaq.com/ website.

After you register, you will be provided with a unique API key, that you should store:

*Note*: Use a `.env` file and put your key in there and `python-dotenv` to access it in this notebook. 

The code below uses a key that was used when generating this project but has since been deleted. Never submit your keys to source control. There is a `.env-example` file in this repository to illusrtate what you need. Copy that to a file called `.env` and use your own api key in that `.env` file. Make sure you also have a `.gitignore` file with a line for `.env` added to it. 

The standard Python gitignore is [here](https://github.com/github/gitignore/blob/master/Python.gitignore) you can just copy that. 

In [1]:
!pip install python-dotenv
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()


print(API_KEY)

xrkM-RYs687E1J8Si-si


Nasdaq Data has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Nasdaq Data API instructions here: https://docs.data.nasdaq.com/docs/in-depth-usage

While there is a dedicated Python package for connecting to the Nasdaq API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [2]:
# First, import the relevant modules
import requests

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url_test = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?limit=1&api_key=' + API_KEY
req_test = requests.get(url_test)

In [4]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
req_test.text

'{"dataset_data":{"limit":1,"transform":null,"column_index":null,"column_names":["Date","Open","High","Low","Close","Change","Traded Volume","Turnover","Last Price of the Day","Daily Traded Units","Daily Turnover"],"start_date":"2000-06-07","end_date":"2020-12-01","frequency":"daily","data":[["2020-12-01",112.2,112.2,111.5,112.0,null,51.0,5703.0,null,null,null]],"collapse":null,"order":null}}'

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [5]:
#url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?&limit=1&api_key=" + API_KEY + "&start_date=2017-01-01&end_date=2017-12-31"
url = "https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json?" + \
      "&start_date=2017-01-01&end_date=2017-12-31&api_key=" + API_KEY
req = requests.get(url)

In [6]:
json_data = req.json()

In [7]:
#Pandas DataFrame to verify the table
import pandas as pd
data_cols = json_data["dataset_data"]["column_names"]
data_rows = json_data["dataset_data"]["data"]
df = pd.DataFrame(data_rows, columns=data_cols)
df.head()

,Date,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
0,2017-12-29,51.76,51.94,51.45,51.76,NaN,34640.0,1792304.0,None,None,None
1,2017-12-28,51.65,51.82,51.43,51.60,NaN,40660.0,2099024.0,None,None,None
2,2017-12-27,51.45,51.89,50.76,51.82,NaN,57452.0,2957018.0,None,None,None
3,2017-12-22,51.05,51.50,50.92,51.32,NaN,71165.0,3641949.0,None,None,None
4,2017-12-21,51.16,51.52,50.90,51.40,NaN,120649.0,6179433.0,None,None,None


In [8]:
open_max=0.0
open_min=float("inf")
for row in json_data["dataset_data"]["data"]:
    if row[1] != None:
       if row[1] > open_max:
          open_max = row[1]
       if row[1] < open_min:
          open_min = row[1]
print("Highest opening price in 2017 was ${0:.2f} and lowest opening price in 2017 was ${1:.2f}." \
      .format(round(open_max, 2), round(open_min, 2)))

Highest opening price in 2017 was $53.11 and lowest opening price in 2017 was $34.00.


In [9]:
diff_max=0.0
for row in json_data["dataset_data"]["data"]:
    if row[1] != None:
       if row[2]-row[3] > diff_max:
          diff_max = row[2]-row[3]
print("The largest change in any one day in 2017 was ${0:.2f}." .format(round(diff_max, 2)))   

The largest change in any one day in 2017 was $2.81.


In [10]:
day_diff_max = {}
close_early =  json_data["dataset_data"]["data"][0][4]

for row in json_data["dataset_data"]["data"][1:]:
    close = row[4]
    day_diff_max[row[0]] = abs(close_early - close)
    close_early = close

day_diff_max_date, day_diff_max_value = max(day_diff_max.items(), key=lambda k:k[1])
print("The largest change between any two days (based on Closing Price) was ${0:.2f} on {1} ." \
      .format(round(day_diff_max_value, 2), (day_diff_max_date)))

The largest change between any two days (based on Closing Price) was $2.56 on 2017-08-08 .


In [11]:
vol_trd= [row[6] for row in json_data["dataset_data"]["data"]]
avg = sum(vol_trd) / len(vol_trd)
print("The average daily trading volume was ${0:.2f} in 2017 ." .format(round(avg, 2)))

The average daily trading volume was $89124.34 in 2017 .


In [12]:
vol_trd= [row[6] for row in json_data["dataset_data"]["data"]]
vol_sorted = sorted(vol_trd) # in ascending order
n=len(vol_sorted)
if n % 2 != 0:
    vol_median = vol_sorted[n // 2]
else:
    vol_median = (vol_sorted[n // 2 - 1] + vol_sorted[n // 2]) / 2

print("The median trading volume was ${0:.2f} in 2017 ." .format(round(vol_median, 2)))

The median trading volume was $76286.00 in 2017 .
